In [ ]:
#!/usr/bin/env python
# coding: utf-8
import sys
sys.path.append('..')
import umbrella_mesh
import elastic_rods
import linkage_vis
from umbrella_mesh import UmbrellaEnergyType
from bending_validation import suppress_stdout as so
from visualization_helper import *

import pipeline_helper, importlib, design_optimization_analysis
with so(): importlib.reload(pipeline_helper)
with so(): importlib.reload(design_optimization_analysis)

from pipeline_helper import UmbrellaOptimizationCallback, allEnergies, allGradientNorms, allDesignObjectives, allDesignGradientNorms, set_joint_vector_field, show_center_joint_normal, show_joint_normal

from design_optimization_analysis import DesignOptimizationAnalysis

import umbrella_optimization
import umbrella_optimization_finite_diff
from umbrella_optimization import OptEnergyType

import numpy as np
import numpy.linalg as la

import pickle, gzip

from configuration import *

### Initialization

In [ ]:
name = 'saddle_5t'
input_path = '../../data/{}.json.gz'.format(name)

io, input_data, target_mesh, curr_um, thickness, target_height_multiplier = parse_input(input_path)
# target_height_multiplier = 1

curr_um = pickle.load(gzip.open('../optimization_experiments/2022_01_25_16_43_saddle_5t/saddle_5t_force_equilibrium_2022_01_25_16_43_target_height_factor_5.0.pkl.gz', 'r'))

#### Pin Rigid Motion

In [ ]:
use_pin = True

driver = curr_um.centralJoint()
jdo = curr_um.dofOffsetForJoint(driver)
fixedVars = (list(range(jdo, jdo + 6)) if use_pin else []) + curr_um.rigidJointAngleDoFIndices()

In [ ]:
import py_newton_optimizer
OPTS = py_newton_optimizer.NewtonOptimizerOptions()
OPTS.gradTol = 1e-8
OPTS.verbose = 1
OPTS.beta = 1e-6
OPTS.niter = 300
OPTS.verboseNonPosDef = False

rod_colors = get_color_field(curr_um, input_data)

# lview = linkage_vis.LinkageViewer(curr_um, width=1024, height=600)
# lview.update(scalarField = rod_colors)
# lview.show()

import mesh
view = linkage_vis.LinkageViewerWithSurface(curr_um, target_mesh, width=1024, height=600)
set_surface_view_options(view, color = 'green', surface_color = 'gray', umbrella_transparent = False, surface_transparent = True)
view.averagedMaterialFrames = True
view.showScalarField(rod_colors)
view.show()

In [ ]:
view.getCameraParams()

view.getSize()

### Deployment

In [ ]:
from equilibrium_solve_analysis import EquilibriumSolveAnalysis
eqays = EquilibriumSolveAnalysis(curr_um)
def eqm_callback(prob, i):
    eqays.record(prob)
    if (i % 2 == 0):
        view.showScalarField(rod_colors)

In [ ]:
configure_umbrella_pre_deployment(curr_um, thickness, target_height_multiplier)

allGradientNorms(curr_um)

In [ ]:
dw = np.zeros(curr_um.numUmbrellas())

In [ ]:
dw[10] = 10
dw[21] = 10

In [ ]:
curr_um.deploymentEnergyWeight = dw

In [ ]:
curr_um.deploymentEnergyWeight

In [ ]:
allEnergies(curr_um)

In [ ]:
curr_um.uniformDeploymentEnergyWeight

In [ ]:
break_input_angle_symmetry(curr_um)

curr_um.attractionWeight = 0
view.showScalarField(rod_colors)
        
with so(): results = umbrella_mesh.compute_equilibrium(curr_um, callback = eqm_callback, options = OPTS, fixedVars = fixedVars, elasticEnergyIncreaseFactorLimit=2.5)


In [ ]:
eqays.plot()

In [ ]:
dw[11] = 10
dw[20] = 10

In [ ]:
curr_um.deploymentEnergyWeight = dw

In [ ]:
curr_um.deploymentEnergyWeight

In [ ]:
allEnergies(curr_um)

In [ ]:
curr_um.uniformDeploymentEnergyWeight

In [ ]:

curr_um.attractionWeight = 0
view.showScalarField(rod_colors)
        
with so(): results = umbrella_mesh.compute_equilibrium(curr_um, callback = eqm_callback, options = OPTS, fixedVars = fixedVars, elasticEnergyIncreaseFactorLimit=2.5)


### Undeployment

In [ ]:
configure_umbrella_undeployment_step_one(curr_um, thickness, target_height_multiplier)

In [ ]:
OPTS.niter = 30

In [ ]:
results = umbrella_mesh.compute_equilibrium(curr_um, callback = eqm_callback, options = OPTS, fixedVars = fixedVars, elasticEnergyIncreaseFactorLimit=2.5)
results.success

In [ ]:
configure_umbrella_undeployment_step_two(curr_um)

In [ ]:
OPTS.niter = 500

In [ ]:
results = umbrella_mesh.compute_equilibrium(curr_um, callback = eqm_callback, options = OPTS, fixedVars = fixedVars, elasticEnergyIncreaseFactorLimit=2.5)
results.success